In [1]:
import copy
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from scipy.linalg import block_diag, inv
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GroupKFold
from scipy.stats import t, zscore

# Autoregressive Mixed Model Data Preprossing

The users need to define the input information like brain regions of interest, which measurement to use.

In [9]:
# input info
ROI_name_list = ['Lamyg', 'Ramyg']
#ROI_name_list = ['L_precentral_thickavg', 'R_precentral_thickavg']
covar_dynamic_name_list = ['Age', 'Sex', 'ICV'] 
# recode sex info!
covar_stable_name_list = ['C1', 'C2','C3', 'C4','APOE4'] 
meausure_time_list = ['sc', '12mo', '24mo']
#n_measure_point = len(meausure_time_list)
#n_timepoint = 3
#slice_length = 3

#genetic information
# Input files
# pycaret package ~
Diagnosis = 'All'
#path_name = '/Users/aliceyang/ADNI1plus2_Longitidinal_Cortical_Subcortical/Subcortical_Volume'
#pheno_file_name = '/ADNI1plus2_MCI_MDS.csv'

#path_name = '/Users/aliceyang/ADNI1plus2_Longitidinal_Cortical_Subcortical/Subcortical_Volume'
#pheno_file_name = '/ADNI1plus2_Dementia_MDS.csv'

path_name = '/Users/aliceyang/ADNI1plus2_Longitidinal_Cortical_Subcortical/Subcortical_Volume_Total_Analysis/Total_MDS'
pheno_file_name = '/ADNI1plus2_total_sorted_MDS_final.csv'

pheno_merge_name = path_name + pheno_file_name

grm_name = path_name + '/ADNI1plus2_GRM_total_sorted_final.csv'

# output path name
output_path_name = '/Users/aliceyang'
file_type_name = '.csv'

# the time point/spatial location ifo should not be coded in the subject ID!
# cohort info
subject_ID_name = 'SubjID'
subject_ID_slice_length = 3

In [10]:
# input info
#ROI_name_list = ['Lhippo', 'Lhippo']
#covar_dynamic_name_list = ['Age', 'Sex'] 
## recode sex info!
#covar_stable_name_list = ['APOE4'] 
#meausure_time_list = ['sc', '12mo', '24mo']
#n_measure_point = len(meausure_time_list)
#n_timepoint = 3
#slice_length = 3

#genetic information
#grm_path = None

# Input files
# pycaret package ~
#path_name = '/Users/aliceyang/ADNI1plus2_Longitidinal_Cortical_Subcortical/Subcortical_Volume'
#pheno_file_name = '/ADNI1plus2_cortical_CN_final_sort.csv'
#pheno_file_name = '/ADNI1plus2_cortical_MCI_final_sort.csv'
#pheno_file_name = '/ADNI1plus2_cortical_Dementia_final_sort.csv'
#pheno_merge_name = path_name + pheno_file_name
#diagnosis_name = 'sAD' # diagnosis name may be extracted from file name

# output path name
#output_path_name = '/Users/aliceyang/result'
#output_file_name = 'res'
#file_type_name = '.csv'

# the time point/spatial location ifo should not be coded in the subject ID!
# cohort info
##subject_ID_name = 'SubjID'
#subject_ID_slice_length = 3

In [5]:
print(pheno_merge_name)

/Users/aliceyang/ADNI1plus2_Longitidinal_Cortical_Subcortical/Subcortical_Volume_Total_Analysis/Total_MDS/ADNI1plus2_total_sorted_MDS_final.csv


# Read files and load column information to the corresponding functions

In [66]:
def load_info_from_file(filename, ROI_name_list, covar_dynamic_name_list, meausure_time_list):
    """
    
    read input csv file containing all phenotype/genotype/covariates such as age and sex.
    
    Parameter(s)
    ----------
    filename: str, csv filename.
    
    Return(s): 
    ----------
    df: dataframe of the csv file.
    
    n: total number of measurements for all subjects.
    
    """
    #print(filename)
    #print(ROI_name_list)
    #print(covar_dynamic_name_list)
    #print(meausure_time_list)
    
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        print('file not found!')
        sys.exit()
    # n = df.shape[0]
    
    dict_roi_name = {}
    for roi in ROI_name_list:
        dict_roi_name[roi] = []
        for tp in meausure_time_list:
            col_name = roi + '_' + tp
            if col_name not in df.columns:
                print(col_name + 'does not exist!')
                sys.exit()
            dict_roi_name[roi].append(col_name)
    #print(dict_roi_name)
    
    dict_dynamic_cov_name = {}
    for cov in covar_dynamic_name_list:
        dict_dynamic_cov_name[cov] = []
        for tp in meausure_time_list:
            col_name = cov + '_' + tp
            if col_name not in df.columns:
                print(col_name + 'does not exist!')
                sys.exit()
            dict_dynamic_cov_name[cov].append(col_name)
    #print(dict_dynamic_cov_name)
    return df, dict_roi_name, dict_dynamic_cov_name

In [67]:
def mapping_matrix_T(measurement):
    """
    
    get mapping matrix to map cross sectional varibles (covariates/ranfom effect correlation) to longitudinal information.
    
    Parameter(s)
    ----------    
    measurement: list, each entry represents the number of measurements for each subject.
    
    Return(s): 
    ----------
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}. 
    
    """
    T = np.expand_dims(np.repeat(1, measurement[0]), axis=1)
    for i in range(1, np.shape(measurement)[0]): 
        T_pre = T
        T = block_diag(T_pre, np.expand_dims(np.repeat(1, measurement[i]), axis = 1))
    return T

In [68]:
def cross_sectional_longit_mapping_function(matrix_T, correlation_matrix):
    """
    
    map function for cross-sectional covariate information or random effect correlation information to longitudinal.
    T=blkdiag{1_n1 , ···,1_nm} 
    
    Parameter(s)
    ----------  
    correlation_matrix: numpy array N x N, the correlation matrix specified for the normal distribution. N number of subjects
    
    matrix_T: numpy array ? x ?, the function to map cross sectional to longitudinal.
    

    
    Return(s): 
    ----------
    longit_correlation: longitudinal correlation matrix mapped from cross sectional 
    
    """
    # use @
    T_correlation = np.matmul(matrix_T, correlation_matrix)
    T_correlation_T_transpose = np.matmul(T_correlation, np.transpose(matrix_T))
    N_total_measurement = np.shape(T_correlation_T_transpose)[0]
    longit_correlation = T_correlation_T_transpose
    #longit_correlation_1d = T_correlation_T_transpose.flatten('F')
    return longit_correlation

In [8]:
def load_grm_info_from_file(n_subject, filename):
    """
       ! optional 
    
    """
    if filename == None:
        grm = np.identity(n_subject)
    else:
        grm = pd.read_csv(filename, header = None)
    return grm

In [9]:
def load_basic_subject_measurement_info_from_file(n_subject, filename, ROI_name_list, ROI_index = None):
    # how to caluculate the number of measurements for each subject?
    if filename == None:
        measurement = df.columns[df.columns.str.startswith(ROI_name_list[ROI_index])]
    else:
        measurement = pd.read_csv(filename, header = None)

In [10]:
pd.read_csv(pheno_merge_name)

,SubjID,DX_sc,DX_12mo,DX_24mo,ICV_sc,ICV_12mo,ICV_24mo,APOE4,Age_sc,Age_12mo,...,Lput_24mo,Rput_24mo,Lpal_24mo,Rpal_24mo,Lhippo_24mo,Rhippo_24mo,Lamyg_24mo,Ramyg_24mo,Laccumb_24mo,Raccumb_24mo
0,002_S_0295,CN,CN,CN,1650465.826,1652724.735,1652220.105,1,84.9,86.0,...,5129.8,4798.0,1493.6,1547.6,3328.2,3406.0,1151.9,1488.8,253.9,315.7
1,002_S_0559,CN,CN,CN,1717852.074,1722711.640,1714273.951,1,79.4,80.5,...,4517.2,4637.4,1288.0,1565.2,3561.2,3632.5,1485.0,1778.1,430.2,395.2
2,002_S_0685,CN,CN,CN,1538868.691,1524683.341,1528160.768,0,89.7,90.8,...,4332.4,5457.0,1543.5,1024.4,3572.8,3335.0,1162.2,1096.4,352.0,219.8
3,002_S_1261,CN,CN,CN,1500525.222,1487317.186,1494543.089,0,71.3,72.5,...,4087.7,3887.0,1411.8,1608.8,2769.7,2936.0,778.0,1070.7,160.8,259.9
4,002_S_4213,CN,CN,CN,1498839.046,1499090.796,1498901.597,0,78.1,79.1,...,3589.5,3962.2,912.3,1083.8,4108.6,3703.1,1235.0,1398.5,475.3,594.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,136_S_0426,Dementia,Dementia,Dementia,1995233.554,1998866.206,2019543.872,1,79.7,80.7,...,6183.6,5859.4,1644.1,1894.0,3068.8,3460.6,1093.0,1369.5,403.6,352.4
701,137_S_0366,Dementia,Dementia,Dementia,1290328.655,1296791.257,1293193.817,0,56.7,57.8,...,3449.1,3535.3,1183.5,1211.8,2625.1,3078.1,749.1,1005.3,269.7,271.7
702,137_S_4211,Dementia,Dementia,Dementia,1350125.957,1332750.512,1333434.641,0,81.0,82.1,...,3820.8,3197.3,1284.2,1242.6,2339.4,1953.6,812.5,752.7,316.1,381.4
703,137_S_4258,Dementia,Dementia,Dementia,1856051.163,1821546.976,1742281.420,1,76.0,77.0,...,4179.3,4436.2,1404.1,2014.5,4081.0,3718.9,1072.3,1124.5,119.7,272.2


In [11]:
# brain measures (left entorhinal thickavg):
#df_pheno_merge, dict_roi_name, dict_dynamic_cov_name = load_info_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)
#print(dict_roi_name)
#print(dict_dynamic_cov_name)

# Calcuate the number of participants
#n_subject = len(df_pheno_merge)

In [12]:
#grm = load_grm_info_from_file(n_subject, filename = grm_name)
#grm = np.array(grm)
df_pheno_merge, dict_roi_name, dict_dynamic_cov_name = load_info_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)

In [13]:
def load_pheno_measurement_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list):
# load phenotype information and get average measures as the longitudinal measure to train/test
    #df_pheno_merge, dict_roi_name, dict_dynamic_cov_name = load_info_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)
    total_measure = []
    longit_measure = []
    for roi, longit_roi in dict_roi_name.items():
        total_measure.append(df_pheno_merge[longit_roi])

    longit_measure = np.mean(total_measure, axis = 0)
    longit_measure = np.expand_dims(longit_measure.flatten('C'), axis = 1)
    #longit_measure_df = pd.DataFrame(longit_measure)
    #longit_measure_df.to_csv('pheno_cMCI_precentral_TH.csv')
    #print(np.shape(longit_measure))
    
    # the number of max time points
    n_timepoint = len(dict_roi_name)
    time_intervals = list(range(n_timepoint))
    count_measure = []
    for roi, col_name in dict_roi_name.items():
        # left and right
        #print(roi)
        #print(col_name)
        count = df_pheno_merge[col_name].notna().sum(axis = 1)
        count_measure.append(count)
    measurement = np.min(count_measure, axis = 0)
    #print(measurement)
    return measurement, longit_measure

In [14]:
# load phenotype information and get average measures as the longitudinal measure to train/test
#total_measure = []
#longit_measure = []
#for roi, longit_roi in dict_roi_name.items():
    #total_measure.append(df_pheno_merge[longit_roi])

#longit_measure = np.mean(total_measure, axis = 0)
#longit_measure = np.expand_dims(longit_measure.flatten('C'), axis = 1)
#print(np.shape(longit_measure))
measurement, longit_measure = load_pheno_measurement_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)

In [15]:
# this is for cross validation group info
def load_groups_from_file(measurement):
    k = 0
    groups = []
    for cnt_meas in measurement:
        for i in range(cnt_meas):
            groups.append(k)
        k = k + 1
    groups = np.array(groups)
    return groups

In [16]:
groups = load_groups_from_file(measurement)

In [17]:
# the number of max time points
#n_timepoint = len(dict_roi_name)

#time_intervals = list(range(n_timepoint))

In [18]:
# the number of measurements per each subject
#subset_df = df[dict_roi_name[0]]

#count_measure = []
#for roi, col_name in dict_roi_name.items():
    # left and right
    #print(roi)
    #print(col_name)
    #count = df_pheno_merge[col_name].notna().sum(axis = 1)
    #count_measure.append(count)
#measurement = np.min(count_measure, axis = 0)

In [21]:
# get mapping matrix T to map from cross sectional to longitudinal
T = mapping_matrix_T(measurement)

# do we need group info for cross validation from meausrement? genetic correlated?
#k = 0
#groups = []
#for cnt_meas in measurement:
    #for i in range(cnt_meas):
        #groups.append(k)
    #k = k + 1
#groups = np.array(groups)
#print(groups)

In [22]:
n_subject = len(df_pheno_merge)
grm = load_grm_info_from_file(n_subject, grm_name)
grm = np.array(grm)
print(grm)

[[ 9.850607e-01  7.943848e-03  8.126090e-04 ...  0.000000e+00
   0.000000e+00  0.000000e+00]
 [ 7.943848e-03  9.976281e-01  5.052495e-03 ...  0.000000e+00
   0.000000e+00  0.000000e+00]
 [ 8.126090e-04  5.052495e-03  1.001683e+00 ...  0.000000e+00
   0.000000e+00  0.000000e+00]
 ...
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  9.871954e-01
   7.460000e-05 -4.621029e-03]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  7.460000e-05
   9.892845e-01 -3.635682e-03]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ... -4.621029e-03
  -3.635682e-03  1.003025e+00]]


# Generate random effect correlation matrix information and extract covariance information


In [23]:
def load_random_effect_info_from_file(grm, measurement, subject_level, T):

    """
    
    get the random effect covariance matrix for each of the ranfom effect (genetic relationship/temporal or spatial effect/site effect/measurement error)
    
    Parameter(s)
    ----------   
    grm: numpy array, input Genetic Relationship Matrix (GRM) for subject-subject correlation.
    
    measurement: list, each entry represents the number of measurements for any subject.
    
    subject_level: numpy array, 
    
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}.
    
    Return(s)
    ----------  
    longit_grm: numpy array, one-dimensional array for longitudinal Genetic Relationship Matrix (GRM).
    
    longit_C1: numpy array,  one-dimensional array for longitudinal correlation matrix corresponding to time intervals between two closest measurements. 
    
    longit_C2: numpy array,  one-dimensional array for longitudinal correlation matrix corresponding to time intervals between two closest measurements.
    
    longit_block_diag_matrix: numpy array,  one-dimensional array for longitudinal block diagnoal matrix 
    
    longit_error: numpy array, one-dimensional array for indepedent measurement erors.
    
    
    """
    
# Create longitudinal GRM 1d info
    grm_arr = np.array(grm)
    longit_grm = cross_sectional_longit_mapping_function(T, grm_arr)
    
# Create covariate matrix input for temporal or spatialy correlated random effect using time intervals or distance functions
# hypothetically  
    if measurement is not None:# Create T always 3 time points
        longit_C1 = np.zeros((measurement[0], measurement[0]))
        longit_C2 = np.zeros((measurement[0], measurement[0]))
        for i in range(measurement[0]):
            for j in range(measurement[0]):
                    if abs(i - j) == 1:
                        longit_C1[i, j] = 1 # rho**abs(i - j) 
                    if abs(i - j) == 2:
                        longit_C2[i, j] = 1 # rho**abs(i - j)
    
        for s in range(1, len(measurement)):
            longit_C1_temp = np.zeros((measurement[s], measurement[s]))
            longit_C2_temp = np.zeros((measurement[s], measurement[s]))
            for i in range(measurement[s]):
                for j in range(measurement[s]):
                    if abs(i - j) == 1:
                        longit_C1_temp[i, j] = 1 # rho**abs(i - j) 
                    if abs(i - j) == 2:
                        longit_C2_temp[i, j] = 1 # rho**abs(i - j)
            # C1 definition
            longit_C1_pre = longit_C1
            longit_C1 = block_diag(longit_C1_pre, longit_C1_temp)
            # C2 definition
            longit_C2_pre = longit_C2
            longit_C2 = block_diag(longit_C2_pre, longit_C2_temp)
        
# Create longitudinal site 1d info using subject level
    subject_level_dummy = pd.get_dummies(subject_level)
    subject_level_cnt = subject_level_dummy.sum(axis=0)
    block_diag_matrix = np.identity(subject_level_cnt[0])
    for i in range(1, np.shape(subject_level_cnt)[0]):
            block_diag_matrix_prev = block_diag_matrix
            block_diag_matrix = block_diag(block_diag_matrix_prev, np.identity(subject_level_cnt[i]))
    longit_block_diag_matrix = cross_sectional_longit_mapping_function(T, block_diag_matrix)
    
# create measurement error 1d info
    longit_error = np.identity(np.sum(measurement))
    return longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error

In [25]:
# get the number of subjects in this study
#len(df_pheno_merge)
#sum(measurement)

In [27]:
def load_covar_info_from_file(df, covar_stable_name_list, dict_dynamic_cov_name, measurement, T):
    
    intercept = np.expand_dims(np.ones(np.sum(measurement)), axis=1)
    if covar_stable_name_list is not None:
        covar_stable_matrix = np.zeros((len(measurement),len(covar_stable_name_list)))
        i = 0
        for cov in covar_stable_name_list:
            cov_value = df[cov]
            #print(cov)
            covar_stable_matrix[:, i] = cov_value
            i = i + 1
        #print(covar_stable_matrix)
        covar_stable_matrix_df = pd.DataFrame(covar_stable_matrix)
        covar_stable_matrix_df.to_csv('geno_cMCI_precentral_TH.csv')
        
        covar_stable_matrix_longit = np.matmul(T, covar_stable_matrix)
        if dict_dynamic_cov_name is None:
            covar_stable_matrix_longit = np.concatenate((intercept,covar_stable_matrix_longit), axis = 1) 
            return covar_stable_matrix_longit
    
    if dict_dynamic_cov_name is not None:
        covar_dynamic_matrix_longit = np.zeros((sum(measurement), len(dict_dynamic_cov_name)))
        j = 0
        for cov in dict_dynamic_cov_name.keys():
            #print(cov)
            cov_longit_value = []
            for cov_longit in dict_dynamic_cov_name[cov]:
                cov_longit_value.append(df[cov_longit])
            # bug?
            cov_longit_value = np.array(cov_longit_value).flatten('F')
            #print(cov_longit_value)
            covar_dynamic_matrix_longit[:, j] = cov_longit_value
            j = j + 1
        if covar_stable_name_list is None:
            covar_dynamic_matrix_longit = np.concatenate((intercept,covar_dynamic_matrix_longit), axis = 1)
            return covar_dynamic_matrix_longit
        
        covar_matrix_longit = np.concatenate((intercept, covar_dynamic_matrix_longit, covar_stable_matrix_longit), axis = 1)
        return covar_matrix_longit

In [28]:
# create subject level to represent scannner/location specific effects
SubjID = df_pheno_merge[subject_ID_name]
subject_level = [ID[:subject_ID_slice_length] for ID in SubjID]

In [30]:
# load ranfom effect information
longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error = load_random_effect_info_from_file(grm, measurement, subject_level, T)
print(longit_C1)
#print(longit_C2)
#print(longit_block_diag_matrix)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [31]:
print(dict_dynamic_cov_name)
print(covar_stable_name_list)

{'Age': ['Age_sc', 'Age_12mo', 'Age_24mo'], 'Sex': ['Sex_sc', 'Sex_12mo', 'Sex_24mo'], 'ICV': ['ICV_sc', 'ICV_12mo', 'ICV_24mo']}
['C1', 'C2', 'C3', 'C4', 'APOE4']


In [32]:
longit_X = load_covar_info_from_file(df_pheno_merge, covar_stable_name_list, dict_dynamic_cov_name, measurement, T)
#longit_X_df = pd.DataFrame(longit_X)
#longit_X_df.to_csv('cov_cMCI_precentral_TH.csv')

# Autoregressive Mixed Model parameter estimation: 2-step method
step 1: project out covariance matrix X including age, sex, SNP encoding.

step 2: run support vector regression to estimate random effect components. 

In [33]:
# 1st step - Create projection matrix to regress out covariance matrix X

def project_covariate_func(X):
    """
    
    get the covariates matrix when the users provide the name of coavariates needed.
    
    Parameter(s)
    ----------   
    X: np.array, longittudinal covariate matrix inluding intercept.
    

    Return(s)
    ----------   
    P: Projection matrix, so that X is projected to 0 matrix through P.
    
    """
    
    XX = np.dot(np.transpose(X), X)
    Z = np.dot(X, inv(XX))
    P = np.diag(np.ones(X.shape[0])) - np.dot(Z, np.transpose(X))
    return P

In [56]:
# 2nd step - main autoregressive linear mixed model estimation function

def ARLMM(Y, X, grm, measurement, subject_level, T):
    """
    
    get the estimated parameters (betas/random effect variances) as the 2nd step of ARLMM.
    
    Parameter(s)
    ----------   
    Y: numpy array, one-dimensional longtidinal measures ordered by the time or spatial points.

    X: numpy array, longitudinal covariate matrix inluding intercept.
    
    grm: numpy array, cross sectional genetic relationship matrix.
    
    measurement: list, each entry represents the number of measurements for any subject.
    
    subject_level: list, ...
    
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}.
    
    

    Return(s)
    ----------   
    rho_est: numeric value, correlation parameter for temporal/spatial correlation matrix.
    
    var_g_est: numeric value, variance of genetic effect.
    
    var_t_est: numeric value, variance of imaging site effect.
    
    var_c_est: numeric value, variance of temporal/spatial correlated effect.
    
    var_e_est: numeric value, variance of meausurement error.
    
    h2_est: numeric value, proportion of genetic effect over total effect.
    
    beta_est: numpy array, beta values for covariance.
    
    """
    
    P = project_covariate_func(X)
    
    longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error = load_random_effect_info_from_file(grm, measurement, subject_level, T)

    project_longit_grm_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_grm),[-1, 1], order = 'F')
    project_longit_C1_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_C1), [-1, 1], order = 'F')
    project_longit_C2_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_C2), [-1, 1], order = 'F')
    project_longit_block_diag_matrix_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_block_diag_matrix), [-1, 1], order = 'F')
    project_longit_error_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_error).flatten('F'), [-1, 1], order = 'F')
    Y_Y_transpose_1d = np.reshape(np.dot(Y, np.transpose(Y)), [-1, 1], order = 'F')
    
    project_longit_X_1d = np.concatenate((project_longit_grm_1d, project_longit_C1_1d, project_longit_C2_1d, project_longit_block_diag_matrix_1d, project_longit_error_1d), axis = 1)
    #clf = SGDRegressor(tol = 1e-3, penalty = 'l2',loss = 'huber', fit_intercept= False)
    clf = SGDRegressor(penalty = 'l2', tol = 1e-3, loss = 'squared_error', fit_intercept= False, epsilon=3000, early_stopping=True)
    clf.fit(project_longit_X_1d, Y_Y_transpose_1d.ravel())
    #clf = MLPRegressor(random_state=1, early_stopping=True).fit(project_longit_X_1d, Y_Y_transpose_1d)
    #clf.fit(project_longit_X_1d, Y_Y_transpose_1d)  
    #n_layers = clf.n_layers_
    #coefs = clf.coefs_
    #print(n_layers)
    #print(coefs)
    
    #from sklearn.linear_model import PassiveAggressiveRegressor
    #clf = PassiveAggressiveRegressor(max_iter=10000, fit_intercept = False, random_state = 0, tol = 1e-3)
    #clf.fit(project_longit_X_1d, Y_Y_transpose_1d)
    
    #print(clf.score(project_longit_X_1d, Y_Y_transpose_1d))
#################3    
    sigma = clf.coef_
    rho_est = max(0.00000000001, min(sigma[2]/sigma[1], 1))
    var_t_est = max(0.5*sigma[1]/rho_est + 0.5*sigma[2]/(rho_est**2), 0.00000000001)
    var_g_est = max(sigma[0], 0.00000000001)
    var_e_est = max(sigma[3] - var_t_est, 0.00000000001)
    var_c_est = max(sigma[4], 0.00000000001)
    h2_est = var_g_est/(var_t_est + var_g_est + var_c_est + var_e_est)
    
    t_error_cov = var_t_est*(np.diag(np.ones(T.shape[0])) + rho_est*longit_C1 + rho_est**2*longit_C2)
    error_cov = var_e_est*np.diag(np.ones(T.shape[0]))
    genetic_cov = var_g_est*longit_grm
    correlation_cov = var_c_est*longit_block_diag_matrix
    total_V = t_error_cov + error_cov + genetic_cov + correlation_cov
    
    Y_new = np.dot(inv(total_V), Y)
    repsonse_new = np.dot(np.transpose(X), Y_new)
    X_new = np.dot(inv(total_V), X)
    predictor_new = np.dot(np.transpose(X), X_new)
    beta_est = np.dot(inv(predictor_new), repsonse_new)
    
    return rho_est, var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est

In [57]:
# run ARLMM main function once and generate best estimates for the whole dataset
rho_est, var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est = ARLMM(longit_measure, longit_X, grm, measurement, subject_level, T)
print(beta_est)
print(h2_est)

[[ 7.78767279e+02]
 [ 9.58698042e-01]
 [-4.67668668e+01]
 [ 3.87534517e-04]
 [ 4.80793910e+03]
 [-2.09885373e+03]
 [ 2.62820964e+02]
 [-1.04692133e+03]
 [-7.77525619e+01]]
0.34034295235048573


# Autoregressive Mixed Model (converter group) 

In [36]:
def ARLMM_late_converter(Y, X, grm, measurement, subject_level, T, stable_parameter):
    # notice for late converter group the correlation matrix used are the same as stable group 
    # but stable parameter is known
    
    """
    
    get the estimated parameters (betas/random effect variances) as the 2nd step of ARLMM.
    
    Parameter(s)
    ----------   
    Y: numpy array, one-dimensional longtidinal measures ordered by the time or spatial points.

    X: numpy array, longitudinal covariate matrix inluding intercept.
    
    grm: numpy array, cross sectional genetic relationship matrix.
    
    measurement: list, each entry represents the number of measurements for any subject.
    
    subject_level: list, ...
    
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}.
    
    stable_parameter: numeric, 
    
    

    Return(s)
    ----------   
    rho_est: numeric value, correlation parameter for temporal/spatial correlation matrix (converter).
    
    var_g_est: numeric value, variance of genetic effect.
    
    var_t_est: numeric value, variance of imaging site effect.
    
    var_c_est: numeric value, variance of temporal/spatial correlated effect.
    
    var_e_est: numeric value, variance of meausurement error.
    
    h2_est: numeric value, proportion of genetic effect over total effect.
    
    beta_est: numpy array, beta values for covariance.
    
    """
    
    P = project_covariate_func(X)
    
    longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error = load_random_effect_info_from_file(grm, measurement, subject_level, T)
    
    project_longit_grm_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_grm),[-1, 1], order = 'F')
    project_longit_C1_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_C1), [-1, 1], order = 'F')
    project_longit_C2_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_C2), [-1, 1], order = 'F')
    project_longit_block_diag_matrix_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_block_diag_matrix), [-1, 1], order = 'F')
    project_longit_error_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_error).flatten('F'), [-1, 1], order = 'F')
    Y_Y_transpose_1d = np.reshape(np.dot(Y, np.transpose(Y)), [-1, 1], order = 'F')
    
    project_longit_X_1d = np.concatenate((project_longit_grm_1d, project_longit_C1_1d, project_longit_C2_1d, project_longit_block_diag_matrix_1d, project_longit_error_1d), axis = 1)
    #clf = SGDRegressor(tol = 1e-3, penalty = 'l2',loss = 'huber', fit_intercept= False)
    clf = SGDRegressor(tol = 1e-3, penalty = 'l2',loss = 'squared_epsilon_insensitive', fit_intercept = False, early_stopping = True)
    clf.fit(project_longit_X_1d, Y_Y_transpose_1d.ravel())
    
    sigma = clf.coef_
    rho_est = max(0.00000000001, min(sigma[2]/sigma[1], 1)) 
    var_t_est = max(0.5*sigma[1]/stable_parameter + 0.5*sigma[2]/(rho_est*stable_parameter), 0.00000000001) 
    var_g_est = max(sigma[0], 0.00000000001) 
    var_e_est = max(sigma[3] - var_t_est, 0.00000000001)
    var_c_est = max(sigma[4], 0.00000000001) 
    h2_est = var_g_est/(var_t_est + var_g_est + var_c_est + var_e_est)
    
    t_error_cov = var_t_est*(np.diag(np.ones(T.shape[0])) + stable_parameter*longit_C1 + rho_est*stable_parameter*longit_C2)
    error_cov = var_e_est*np.diag(np.ones(T.shape[0]))
    genetic_cov = var_g_est*longit_grm
    correlation_cov = var_c_est*longit_block_diag_matrix
    total_V = t_error_cov + error_cov + genetic_cov + correlation_cov
    
    Y_new = np.dot(inv(total_V), Y)
    repsonse_new = np.dot(np.transpose(X), Y_new)
    X_new = np.dot(inv(total_V), X)
    predictor_new = np.dot(np.transpose(X), X_new)
    beta_est = np.dot(inv(predictor_new), repsonse_new)
    
    return rho_est, var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est

In [40]:
def ARLMM_total_converter(Y, X, grm, measurement, subject_level, beta_late_convert):
    """
    
    get the estimated parameters (betas/random effect variances) as the 2nd step of ARLMM.
    
    Parameter(s)
    ----------   
    Y: numpy array, ...

    X: numpy array, ...
    
    grm: numpy array, ...
    
    measurement: list, ...
    
    subject_level: list, ...
    
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}.
    
    DX_label: new label of diagnosis, for example 0 = 'early converter' and 1 = 'late converter'
    
    """
    N = np.shape(grm)[0]
        
    longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error = load_random_effect_info_from_file(grm, measurement, subject_level, T)
    
    # Accurate Method 
    # t_error_cor = np.diag(np.ones(T.shape[0])) + stable_parameter0*longit_C1 + rho_est*stable_parameter0*longit_C2
    # t_error_cor = np.diag(np.ones(T.shape[0])) + rho_est*longit_C1 + rho_est*stable_parameter1*longit_C2            
    early_convert_index = np.where(DX_label == 0)[0]
    late_convert_index = np.where(DX_label == 1)[0]

    early_convert_X, early_convert_Y, early_convert_measurement = X[early_convert_index], Y[early_convert_index], measurement[early_convert_index]
    early_convert_grm = grm[early_convert_index, :]
    early_convert_grm = early_convert_grm[:, early_convert_index]
    early_convert_T = np.zeros((np.shape(early_convert_index)[0], np.shape(early_convert_index)[0]))
    early_convert_T = mapping_matrix_T(early_convert_measurement)
    longit_grm_early_convert, longit_C1_early_convert, longit_C2_early_convert, longit_block_diag_matrix_early_convert, longit_error_early_convert = load_random_effect_info_from_file(grm_early_convert, measurement_early_convert, subject_level_early_convert, T_early_convert)
    
    late_convert_X, late_convert_Y, late_convert_measurement = X[late_convert_index], Y[late_convert_index], measurement[late_convert_index]
    late_convert_grm = grm[late_convert_index, :]
    late_convert_grm = late_convert_grm[:, late_convert_index]
    late_convert_T = np.zeros((np.shape(late_convert_index)[0], np.shape(late_convert_index)[0]))
    late_convert_T = mapping_matrix_T(late_convert_measurement) 
    longit_grm_late_convert, longit_C1_late_convert, longit_C2_late_convert, longit_block_diag_matrix_late_convert, longit_error_late_convert = load_random_effect_info_from_file(grm_early_convert, measurement_early_convert, subject_level_early_convert, T_early_convert)
    
    # concate matrices from two groups, we have X and Y concatenated
    longit_grm = block_diag(early_convert_grm, late_convert_grm)
    T = block_diag(early_convert_T, late_convert_T)
    longit_error = block_diag(longit_error_early_convert, longit_error_late_convert)
    longit_block_diag_matrix_error = block_diag(longit_block_diag_matrix_early_convert, longit_block_diag_matrix_late_convert)
    
    late_convert_C = np.ones(late_convert_T.shape[0]) + beta_late_convert*longit_C1_late_convert + beta_late_convert*theta_est_late_convert*longit_C2_late_convert
    early_convert_C = np.ones(early_convert_T.shape[0]) + beta_late_convert*longit_C1_late_convert + beta_late_convert*theta_est_late_convert*longit_C2_late_convert
    C = block_diag(early_convert_C, late_convert_C)
    
    # projection matrix P
    P = project_covariate_func(X)
    
    # project out covariates X
    project_longit_grm_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_grm),[-1, 1], order = 'F')
    project_longit_C_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_C), [-1, 1], order = 'F')
    project_longit_block_diag_matrix_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_block_diag_matrix), [-1, 1], order = 'F')
    project_longit_error_1d = np.reshape(cross_sectional_longit_mapping_function(P, longit_error).flatten('F'), [-1, 1], order = 'F')
    Y_Y_transpose_1d = np.reshape(np.dot(Y, np.transpose(Y)), [-1, 1], order = 'F')
    
    project_longit_X_1d = np.concatenate((project_longit_grm_1d, project_longit_C_1d, project_longit_block_diag_matrix_1d, project_longit_error_1d), axis = 1)

    clf = SGDRegressor(tol = 1e-3, penalty = 'l2',loss = 'huber', fit_intercept= False)
    clf.fit(project_longit_X_1d, Y_Y_transpose_1d.ravel())

    sigma = clf.coef_
    var_t_est = max(sigma[2], 0.00000000001) 
    var_g_est = max(sigma[0], 0.00000000001) 
    var_e_est = max(sigma[3], 0.00000000001)
    var_c_est = max(sigma[1], 0.00000000001) 
    h2_est = var_g_est/(var_t_est + var_g_est + var_c_est + var_e_est)
    
    t_error_cov = var_t_est*longit_block_diag_matrix
    error_cov = var_e_est*longit_error
    genetic_cov = var_g_est*longit_grm
    correlation_cov = var_c_est*longit_block_diag_matrix
    total_V = t_error_cov + error_cov + genetic_cov + correlation_cov
    
    Y_new = np.dot(inv(total_V), Y)
    repsonse_new = np.dot(np.transpose(X), Y_new)
    X_new = np.dot(inv(total_V), X)
    predictor_new = np.dot(np.transpose(X), X_new)
    beta_est = np.dot(inv(predictor_new), repsonse_new)
    return var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est
    # get estimates of all variance components

# Autoregressive Mixed Model  evaluation and prediction

In [37]:
def predict(X, beta):
    """
    
    predict longitudinal phenotype measures given estimated beta values.
    
    Parameter(s)
    ----------   
    X: np.array, longittudinal covariate matrix inluding intercept [test data].
    

    Return(s)
    ----------   
    y_pred: np.array, predicted 
    
    """
    X = np.array(X)
    y_pred = np.matmul(X, beta) # 2d numpy array
    return y_pred

In [38]:
def evaluate(y_pred, y_true):
    """
    
    evaluate the predicted vaues with true values on the test data.
    
    Parameter(s)
    ----------   
    y_pred: np.array, predicted phenotype values.
    y_true: np.array, real phenotype values.
    
    Return(s)
    ----------   
    mse: numeric value, mean squared error.
    rmse: numeric value, root mean squared error.
    mae: numeric value, mean absolute error.
    r2: numeric value, coefficient of determination.
    
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, r2 

In [55]:
# we print t value here
def booostrap_stats(booostrap_sample, hypothesis_testing_type = 'two-sided', significance_level = 0.05, df = 100):
    """
    get the p-value for testing SNP of interest
    
    """
    CI_upper = np.quantile(booostrap_sample, 1 - significance_level/2)
    CI_lower = np.quantile(booostrap_sample, significance_level/2)
    t_stats = np.mean(booostrap_sample)/np.std(booostrap_sample)
    print(CI_upper)
    print(CI_lower)
    if hypothesis_testing_type == 'left-sided' or 'right-sided':
        p_value = (1-t.cdf(abs(t_stats), df))
    if hypothesis_testing_type == 'two-sided':
        p_value =2*(1-t.cdf(abs(t_stats), df))
    p_value = p_value[0]
    return CI_lower, CI_upper, p_value, t_stats

In [40]:
def group_kfold_cross_validation(n_splits, longit_measure, longit_X, grm, measurement, subject_level, T, diagnosis_group = None, stable_parameter = 1.0):
    """
    perform K-fold cross validation.
    
    Parameter(s)
    ----------   
    n_splits: numeric value, the number of folds for cross validation.
    longit_measure: numpy array, one-dimensional left longtidinal measures ordered by the time or spatial points.
    
    longit_X: numpy array, longtudinal covariates.
    grm: numpy array, cross sectional genetic relationship matrix (GRM).
    measurement: numpy array, number of measurements for each subject (cross sectional).
    subject_level: numpy array, number of measurements for each subject (cross sectional).
    T: numpy array, the mapping matrix T, T=blkdiag{1_n1 , ···,1_nm}.
    
    Return(s)
    ----------   
    TRAIN_RMSE: numeric value, RMSE for training data.
    TRAIN_MAE: numeric value, MAE for traning data.
    TRAIN_R2: numeric value, R2 for traning data.
    TEST_RMSE: numeric value, RMSE for test data.
    TEST_MAE: numeric value, MAE for test data.
    TEST_R2: numeric value, R2 for test data.
    
    """
    group_kfold = GroupKFold(n_splits)
    TRAIN_MSE = []
    TRAIN_RMSE = []
    TRAIN_MAE = []
    TRAIN_R2 = []
    TRAIN_BETA = []
    TRAIN_RHO = []
    TEST_RHO = []

    TEST_MSE = []
    TEST_RMSE = []
    TEST_MAE = []
    TEST_r2 = []
    TRAIN_H2 = []
    TEST_H2 = []
    for i, (train_index, test_index) in enumerate(group_kfold.split(longit_X, longit_measure, groups)): 
        #print(f" Train: index={train_index}, group={groups[train_index]}") 
        #print(f" Test: index={test_index}, group={groups[test_index]}") 
        # train_index and test_index are for longit - groups_train_index and groups_test_index are for cross sectional 
        groups_train_index = [] 
        groups_test_index = [] 
        for i in groups[train_index]: 
            if i not in groups_train_index: 
                groups_train_index.append(i)

        for j in groups[test_index]:
            if j not in groups_test_index:
                groups_test_index.append(j)
        #print(groups_test_index)
        train_longit_measure = longit_measure[train_index]
        train_longit_X = longit_X[train_index]
        train_grm = grm[groups_train_index, :] # wrong
        train_grm = train_grm[:, groups_train_index] # wrong
        train_subject_level = [subject_level[i] for i in groups_train_index]
        train_measurement = measurement[groups_train_index]
        train_T = np.zeros((np.shape(train_index)[0], np.shape(groups_train_index)[0]))
        train_T = mapping_matrix_T(train_measurement)

        test_longit_measure = longit_measure[test_index]
        test_longit_X = longit_X[test_index]
        test_grm = grm[groups_test_index, :] # wrong
        test_grm = test_grm[:, groups_test_index] # wrong
        test_subject_level = [subject_level[i] for i in groups_test_index]
        test_measurement = measurement[groups_test_index]
        test_T = np.zeros((np.shape(test_index)[0], np.shape(groups_test_index)[0]))
        test_T = mapping_matrix_T(test_measurement)
        #rho_est, var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est
        if diagnosis_group == 'stable':
            train_rho, train_var_g, train_var_t, train_var_c, train_var_e, train_h2, train_beta = ARLMM(train_longit_measure, train_longit_X, train_grm, train_measurement, train_subject_level, train_T)
        elif diagnosis_group == 'late_converter':
            train_rho, train_var_g, train_var_t, train_var_c, train_var_e, train_h2, train_beta = ARLMM_late_converter(train_longit_measure, train_longit_X, train_grm, train_measurement, train_subject_level, train_T, stable_parameter)

        train_mse, train_rmse, train_mae, train_r2 = evaluate(predict(train_longit_X, train_beta), train_longit_measure)
        test_mse, test_rmse, test_mae, test_r2 = evaluate(predict(test_longit_X, train_beta), test_longit_measure)
        #print(np.shape(predict(train_longit_X, train_beta)))
        #predict(train_longit_X, train_beta)
        #longit_test_y_pred.append(predict(test_longit_X, train_beta))
        TRAIN_MSE.append(train_mse)
        TRAIN_RMSE.append(train_rmse)
        TRAIN_MAE.append(train_mae)
        TRAIN_R2.append(train_r2)
        TRAIN_BETA.append(train_beta)
        #TRAIN_RHO.append(train_rho)
        TRAIN_H2.append(train_h2)
        
        TEST_MSE.append(test_mse)
        TEST_RMSE.append(test_rmse)
        TEST_MAE.append(test_mae)
        TEST_r2.append(test_r2)
    #TRAIN_RHO_MEAN = np.mean(TRAIN_RHO)

    # boostrap to stimate p value
    #if True: 
    #if n_splits == n_subject: 
        #BETA_SNP = np.array(TRAIN_BETA)[:, -1]
        #CI_lower_SNP, CI_upper_SNP, p_value_SNP = booostrap_stats(BETA_SNP, hypothesis_testing_type = 'two-sided', significance_level = 0.05, df = np.shape(train_index))# Estimate beta values
        #return BETA_SNP, CI_lower_SNP, CI_upper_SNP, p_value_SNP
    #else:
        #TRAIN_RMSE_MEAN = np.mean(TRAIN_RMSE, axis=0)
        #TRAIN_MAE_MEAN = np.mean(TRAIN_MAE, axis=0)
        #TRAIN_R2_MEAN = np.mean(TRAIN_R2, axis=0)
        #TRAIN_RHO_MEAN = np.mean(TRAIN_RHO, axis=0)
    
        #TEST_MSE_MEAN = np.mean(TEST_MSE, axis=0)
        #TEST_RMSE_MEAN = np.mean(TEST_RMSE, axis=0)
        #TEST_MAE_MEAN = np.mean(TEST_MAE, axis=0)
        #TEST_R2_MEAN = np.mean(TEST_R2, axis=0)
        #TEST_RHO_MEAN = np.mean(TEST_RHO, axis=0)   
        #print('MEAN_TRAIN_RMSE:', np.mean(TRAIN_RMSE), 'MEAN_TRAIN_MAE:', np.mean(TRAIN_MAE), 'MEAN_TRAIN_R2:', np.mean(TRAIN_R2))
        #print('MEAN_TEST_RMSE:', np.mean(TEST_RMSE), 'MEAN_TEST_MAE:', np.mean(TEST_MAE), 'MEAN_TEST_R2:', np.mean(TEST_R2))
    
    # Estimate beta values
    train_beta_estimate = np.mean(TRAIN_BETA, axis=0)
    beta_snp_sample = np.array(TRAIN_BETA)[:, -1]
    # calculate z score
    CI_lower_SNP, CI_upper_SNP, p_value_SNP, z_score_SNP = booostrap_stats(beta_snp_sample, hypothesis_testing_type = 'two-sided', significance_level = 0.05, df = np.shape(train_index))# Estimate beta values 
    #print('SNP z score:', z_score_SNP)
    # calculate p value
    #print('SNP p value:', p_value_SNP)
    
    # h2 raw values
    train_h2_estimate = np.mean(TRAIN_H2) 
    # h2 p values
    CI_lower_h2, CI_upper_h2, p_value_h2, z_score_h2 = booostrap_stats(TRAIN_H2, hypothesis_testing_type = 'two-sided', significance_level = 0.05, df = np.shape(train_index))# Estimate beta values 
    #print('h2 estimate:', train_h2_estimate)
    #print('h2 p value:', p_value_h2)
    test_rmse_estimate = np.mean(TEST_RMSE, axis=0)
    test_rmse_std = np.std(TEST_RMSE, axis=0)
    #print('test_rmse_estimate:', test_rmse_estimate)
    #print('test_rmse_std:', test_rmse_std)
    
    # MAE raw values, mean MAE, and std MAE
    test_mae_estimate = np.mean(TEST_MAE, axis=0)
    test_mae_std = np.std(TEST_MAE, axis=0)
    #print('test_mae_estimate:', test_mae_estimate)
    #print('test_mae_std:', test_mae_std)
    
    # R2 raw values, mean R2, and std R2
    test_r2_estimate = np.mean(TEST_r2, axis=0)
    test_r2_std = np.std(TEST_r2, axis=0)
    #print('test_r2_estimate:', test_r2_estimate)
    #print('test_r2_std:', test_r2_std)    
    return p_value_SNP, z_score_SNP, TRAIN_H2, train_h2_estimate, p_value_h2, TEST_RMSE, test_rmse_estimate, test_rmse_std, TEST_MAE, test_mae_estimate, test_mae_std, TEST_r2, test_r2_estimate, test_r2_std

# Save results and model

In [41]:
n_splits = 5
#n_splits = n_subject
p_value_SNP, z_score_SNP, TRAIN_H2, train_h2_estimate, p_value_h2, TEST_RMSE, test_rmse_estimate, test_mae_std, TEST_MAE, test_mae_estimate, test_mae_std, TEST_r2, test_r2_estimate, test_mae_std = group_kfold_cross_validation(n_splits, longit_measure, longit_X, grm, measurement, subject_level, T, diagnosis_group = 'stable', stable_parameter = 1.0)

In [42]:
#SNP_stats_names = ['p_value_SNP', 'z_score_SNP']
#SNP_stats_values = [p_value_SNP, z_score_SNP]

#h2_stats_names = ['h2 estimate', 'h2 p value']
#h2_stats_values = [train_h2_estimate, p_value_h2]

#rmse_stats_names = ['rmse_estimate', 'rmse_std']
#rmse_stats_values = [test_rmse_estimate, test_rmse_std]

#mae_stats_names = ['mae_estimate', 'mae_std']
#mae_stats_values = [test_mae_estimate, test_mae_std]

#r2_stats_names = ['r2_estimate', 'r2_std']
#r2_stats_values = [test_r2_estimate, test_r2_std]

#all_res_names = ['all_h2', 'all RMSE', 'all MAE', 'all r2']
#all_res_values = [TRAIN_H2, TEST_RMSE, TEST_MAE, TEST_r2]

In [43]:
#dict_SNP = dict(zip(SNP_stats_names, SNP_stats_values))
#dict_h2 = dict(zip(h2_stats_names, h2_stats_values))
#dict_rmse = dict(zip(rmse_stats_names, rmse_stats_values))
#dict_r2 = dict(zip(r2_stats_names, r2_stats_values))

In [44]:
#dict_res = dict_SNP | dict_h2 | dict_rmse | dict_r2

In [45]:
#dict_all_res = dict(zip(all_res_names, all_res_values))

In [46]:
#print(dict_res)
#df_stats = pd.DataFrame(dict_res, index=[0])
#print(df_stats)
#df_all_stats = pd.DataFrame(dict_all_res)
#print(df_all_stats)

In [47]:
#df.to_csv(output_path_name + '/' + 'ARLMM_' + ROI_name_list[0][1:] +'_' +'APOE4' +'_' + Diagnosis + file_type_name)

# run all ROIs

In [48]:
L_ROI = ['Laccumb', 'Lamyg', 'Lcaud', 'Lhippo', 'Lpal', 'Lput', 'Lthal']
R_ROI = ['Raccumb', 'Ramyg', 'Rcaud', 'Rhippo', 'Rpal', 'Rput', 'Rthal']

In [49]:
def final_calculation(L_ROI, R_ROI, covar_dynamic_name_list, meausure_time_list, covar_stable_name_list, n_splits):# brain measures (left entorhinal thickavg):
    df_pheno_merge, dict_roi_name_L, dict_dynamic_cov_name = load_info_from_file(pheno_merge_name, L_ROI, covar_dynamic_name_list, meausure_time_list)
    df_pheno_merge, dict_roi_name_R, dict_dynamic_cov_name = load_info_from_file(pheno_merge_name, R_ROI, covar_dynamic_name_list, meausure_time_list)
    #print(dict_roi_name_L)
    #print(dict_roi_name_R)
    N_ROI = len(L_ROI)
    final_result = []
    #print(dict_roi_name)
    #print(dict_dynamic_cov_name)
    #print(N_ROI)   
    for i in range(N_ROI):
        L_R_ROI_name_list = [L_ROI[i], R_ROI[i]]
        #template: measurement, longit_measure = load_pheno_measurement_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)
        measurement, longit_measure = load_pheno_measurement_from_file(pheno_merge_name, L_R_ROI_name_list, covar_dynamic_name_list, meausure_time_list)
        #print(longit_measure)
        
        # mapping matrix
        T = mapping_matrix_T(measurement)
        
        # create subject level to represent scannner/location specific effects
        SubjID = df_pheno_merge[subject_ID_name]
        subject_level = [ID[:subject_ID_slice_length] for ID in SubjID]
        
        # Calcuate the number of participants
        # load Genetic Relationship Matrix
        n_subject = len(df_pheno_merge)
        grm = load_grm_info_from_file(n_subject, filename = grm_name)
        grm = np.array(grm)    
        #print(grm)
        
        # longitudinal measures
        measurement, longit_measure = load_pheno_measurement_from_file(pheno_merge_name, ROI_name_list, covar_dynamic_name_list, meausure_time_list)
        
        # load ranfom effect information
        longit_grm, longit_C1, longit_C2, longit_block_diag_matrix, longit_error = load_random_effect_info_from_file(grm, measurement, subject_level, T)
        #print(longit_C1)
        #print(longit_C2)
        #print(longit_block_diag_matrix)
        
        longit_X = load_covar_info_from_file(df_pheno_merge, covar_stable_name_list, dict_dynamic_cov_name, measurement, T)
        #longit_X_df = pd.DataFrame(longit_X)
        #longit_X_df.to_csv('cov_cMCI_precentral_TH.csv')
        
        # one time estimate
        rho_est, var_g_est, var_t_est, var_c_est, var_e_est, h2_est, beta_est = ARLMM(longit_measure, longit_X, grm, measurement, subject_level, T)
        #print(beta_est)
        #print(h2_est)  
        
        # cross validation
        #n_splits = 5
        p_value_SNP, z_score_SNP, TRAIN_H2, train_h2_estimate, p_value_h2, TEST_RMSE, test_rmse_estimate, test_rmse_std, TEST_MAE, test_mae_estimate, test_mae_std, TEST_r2, test_r2_estimate, test_r2_std = group_kfold_cross_validation(n_splits, longit_measure, longit_X, grm, measurement, subject_level, T, diagnosis_group = 'stable', stable_parameter = 1.0)

        snp_p_value.append(p_value_SNP)
        snp_z_score.append(z_score_SNP)
        # h2 value
        h2.append(train_h2_estimate)
        h2_p_value.append(p_value_h2)
        # predictive stats 
        test_rmse.append(test_rmse_estimate)
        test_rmse_std_value.append(test_rmse_std)
        
        test_mae.append(test_mae_estimate)
        test_mae_std_value.append(test_rmse_std)
        
        test_r2.append(test_r2_estimate)  
        test_r2_std_value.append(test_r2_std)
    return snp_z_score, snp_p_value, h2, h2_p_value, test_rmse, test_rmse_std_value, test_mae, test_mae_std_value, test_r2, test_r2_std_value

In [58]:
snp_p_value = []
snp_z_score = []

h2 = []
h2_p_value = []

test_rmse = []
test_rmse_std_value = []
test_mae = []
test_mae_std_value = []
test_r2 = []
test_r2_std_value = []
n_splits = 5
snp_z_score_all_ROI, snp_p_value_all_ROI, h2_all_ROI, h2_p_value_all_ROI, test_rmse_all_ROI, test_rmse_std_all_ROI, test_mae_all_ROI, test_mae_std_all_ROI, test_r2_all_ROI, test_r2_std_all_ROI = final_calculation(L_ROI, R_ROI, covar_dynamic_name_list, meausure_time_list, covar_stable_name_list, n_splits)

-72.6087709685869
-82.38654562433972
0.5417821243076115
0.24049949793636768
-71.66003666539692
-80.1008191141736
0.39422427191006715
0.030155787103812254
-67.69489795237345
-81.58668080453683
0.3664267361119898
0.04402306904102971
-71.9976786816529
-81.79964361148261
0.5445731056792308
0.25668906404852054
-71.75547812228331
-78.3116140942996
0.3755898727588333
1.7496092714482962e-22
-72.38961507448873
-81.33616381732115
0.5070856949636545
0.05768696736184206
-67.30940985632262
-82.27061879408004
0.41487392166782133
0.025898567933151147


In [52]:
snp_p_value = []
snp_z_score = []

h2 = []
h2_p_value = []

test_rmse = []
test_rmse_std_value = []
test_mae = []
test_mae_std_value = []
test_r2 = []
test_r2_std_value = []
n_splits = 5
snp_z_score_cv, snp_p_value_cv, h2_cv, h2_p_value_cv, test_rmse_cv, test_rmse_std_cv, test_mae_cv, test_mae_std_cv, test_r2_cv, test_r2_std_cv = final_calculation(L_ROI, R_ROI, covar_dynamic_name_list, meausure_time_list, covar_stable_name_list, n_splits)

In [59]:
print('snp_z_score:', snp_z_score_all_ROI)
print('snp_p_value:', snp_p_value_all_ROI)
print('h2:', h2_all_ROI)
print('h2_p_value:', h2_p_value_all_ROI)
print('test_rmse:', test_rmse_cv)
print('test_rmse_std:', test_rmse_std_cv)
print('test_mae:', test_mae_cv)
print('test_mae_std:', test_mae_std_cv)
print('test_r2:', test_r2_cv)
print('test_r2_std:', test_r2_std_cv)

snp_z_score: [-18.945657531659162, -23.567059875999174, -14.000242517839677, -20.760942949288324, -28.697624631190596, -22.165779549788184, -13.81195360753005]
snp_p_value: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
h2: [0.3958208345957287, 0.27927344077807803, 0.25392159405509507, 0.3600519533687058, 0.13173837405940123, 0.33200280993738407, 0.25315903592275896]
h2_p_value: [0.000677130090491529, 0.051685832153836486, 0.03788672985290287, 0.0013067667744841582, 0.42596340463337223, 0.05776803131057906, 0.06998770779104535]
test_rmse: [252.20983133908226, 252.2345965717171, 252.23524686234182, 252.24802128462207, 252.23072061322355, 252.23892442013366, 252.24855381650787]
test_rmse_std: [9.168946540457362, 9.156383566811035, 9.136756555181503, 9.192914542967708, 9.136164628671587, 9.153500961076126, 9.177982176354305]
test_mae: [201.0097275453027, 201.02622538154657, 201.02170378405253, 201.03225988057116, 201.01590700932877, 201.04508043255345, 201.03763119207073]
test_mae_std: [9.1689465404

In [60]:
ROI_name = ['accumbens', 'amygdala', 'caudate', 'hippocampus', 'pallidum', 'putamen', 'thalamus']

dict_snp_z_score_value = dict(zip(ROI_name, snp_z_score_all_ROI))
dict_snp_p_value = dict(zip(ROI_name, snp_p_value_all_ROI))

dict_h2_value = dict(zip(ROI_name, h2_all_ROI))
dict_h2_p_value = dict(zip(ROI_name, h2_p_value_all_ROI))

dict_rmse_value = dict(zip(ROI_name, test_rmse_cv))
dict_rmse_std_value = dict(zip(ROI_name, test_rmse_std_cv))

dict_mae_value = dict(zip(ROI_name, test_mae_cv))
dict_mae_std_value = dict(zip(ROI_name, test_mae_std_cv))

dict_r2_value = dict(zip(ROI_name, test_r2_cv))
dict_r2_std_value = dict(zip(ROI_name, test_r2_std_cv))
print(dict_snp_z_score_value)
print(dict_snp_p_value)
print(dict_h2_value)
print(dict_h2_p_value)

print(dict_rmse_value)
print(dict_rmse_std_value)

print(dict_mae_value)
print(dict_mae_std_value)

print(dict_r2_value)
print(dict_r2_std_value)

{'accumbens': -18.945657531659162, 'amygdala': -23.567059875999174, 'caudate': -14.000242517839677, 'hippocampus': -20.760942949288324, 'pallidum': -28.697624631190596, 'putamen': -22.165779549788184, 'thalamus': -13.81195360753005}
{'accumbens': 0.0, 'amygdala': 0.0, 'caudate': 0.0, 'hippocampus': 0.0, 'pallidum': 0.0, 'putamen': 0.0, 'thalamus': 0.0}
{'accumbens': 0.3958208345957287, 'amygdala': 0.27927344077807803, 'caudate': 0.25392159405509507, 'hippocampus': 0.3600519533687058, 'pallidum': 0.13173837405940123, 'putamen': 0.33200280993738407, 'thalamus': 0.25315903592275896}
{'accumbens': 0.000677130090491529, 'amygdala': 0.051685832153836486, 'caudate': 0.03788672985290287, 'hippocampus': 0.0013067667744841582, 'pallidum': 0.42596340463337223, 'putamen': 0.05776803131057906, 'thalamus': 0.06998770779104535}
{'accumbens': 252.20983133908226, 'amygdala': 252.2345965717171, 'caudate': 252.23524686234182, 'hippocampus': 252.24802128462207, 'pallidum': 252.23072061322355, 'putamen': 2

In [61]:
df_snp_z_score_value = pd.DataFrame(dict_snp_z_score_value, index = [Diagnosis])
df_snp_p_value = pd.DataFrame(dict_snp_p_value, index = [Diagnosis])

df_h2_value = pd.DataFrame(dict_h2_value, index = [Diagnosis])
df_h2_p_value = pd.DataFrame(dict_h2_p_value, index = [Diagnosis])

df_rmse_value = pd.DataFrame(dict_rmse_value, index = [Diagnosis])
df_rmse_std_value = pd.DataFrame(dict_rmse_std_value, index = [Diagnosis])

df_mae_value = pd.DataFrame(dict_mae_value, index = [Diagnosis])
df_mae_std_value = pd.DataFrame(dict_mae_std_value, index = [Diagnosis])

df_r2_value = pd.DataFrame(dict_r2_value, index = [Diagnosis])
df_r2_std_value = pd.DataFrame(dict_r2_std_value, index = [Diagnosis])

print(df_r2_value)
print(df_r2_std_value)

     accumbens  amygdala   caudate  hippocampus  pallidum   putamen  thalamus
All     0.1006  0.100427  0.100419     0.100339  0.100452  0.100396  0.100332
     accumbens  amygdala   caudate  hippocampus  pallidum   putamen  thalamus
All   0.041078  0.040924  0.040891     0.040937  0.040871  0.040904   0.04093


In [62]:
df_snp_z_score_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'snp_z_score' + '_' + Diagnosis + file_type_name)
df_snp_p_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'snp_p_value' + '_' + Diagnosis + file_type_name)

df_h2_value.to_csv(output_path_name + '/' + 'ARLMM' + '_' + 'h2_value' + '_' + Diagnosis + file_type_name)
df_h2_p_value.to_csv(output_path_name + '/' + 'ARLMM' + '_' + 'h2_p_value' + '_' + Diagnosis + file_type_name)

In [63]:

df_rmse_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'RMSE' + '_' + Diagnosis + file_type_name)
df_rmse_std_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'RMSE_std' + '_' + Diagnosis + file_type_name)

df_mae_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'MAE' + '_' + Diagnosis + file_type_name)
df_mae_std_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'MAE_std' + '_' + Diagnosis + file_type_name)

df_r2_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'R2' + '_' + Diagnosis + file_type_name)
df_r2_std_value.to_csv(output_path_name + '/' + 'ARLMM' +'_' + 'R2_std' + '_' + Diagnosis + file_type_name)

# Linear Regression

In [68]:
from sklearn.model_selection import KFold
import statsmodels.api as sm

In [65]:
# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [66]:
# Initialize lists to store evaluation metrics
mse_train_scores = []
mae_train_scores = []
r2_train_scores = []

# Initialize lists to store evaluation metrics
mse_test_scores = []
mae_test_scores = []
r2_test_scores = []

In [69]:
for train_index, test_index in kf.split(longit_X):
    X_train, X_test = longit_X[train_index], longit_X[test_index]
    y_train, y_test = longit_measure[train_index], longit_measure[test_index]
    
    # Fit the model
    model = sm.OLS(y_train, X_train)
    results = model.fit()
    
    # Predict on the test set
    y_pred = results.predict(X_test)
    y_train_pred = results.predict(X_train)
    # Calculate evaluation metrics
    mse_test_scores.append(mean_squared_error(y_test, y_pred))
    mae_test_scores.append(mean_absolute_error(y_test, y_pred))
    r2_test_scores.append(r2_score(y_test, y_pred))
    
    mse_train_scores.append(mean_squared_error(y_train, y_train_pred))
    mae_train_scores.append(mean_absolute_error(y_train, y_train_pred))
    r2_train_scores.append(r2_score(y_train, y_train_pred))

In [70]:
print(np.sqrt(np.mean(mse_train_scores)))
print(np.mean(mae_train_scores))
print(np.mean(r2_train_scores))

print(np.sqrt(np.mean(mse_test_scores)))
print(np.mean(mae_test_scores))
print(np.mean(r2_test_scores))

249.30824361591
198.77728463738487
0.1254549613298589
251.05181820448752
199.80529359635946
0.11150913367996437
